# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Apr 2, 11:59pm](https://github.com/UBC-CS/cpsc330-2023W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W2/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [4]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [5]:
glove_wiki_vectors.most_similar("food")

[('foods', 0.7469059824943542),
 ('supplies', 0.7264691591262817),
 ('products', 0.7225049138069153),
 ('meat', 0.7138239741325378),
 ('supply', 0.6732638478279114),
 ('feed', 0.670415461063385),
 ('medicines', 0.6687098145484924),
 ('meals', 0.6630423069000244),
 ('coffee', 0.6627735495567322),
 ('goods', 0.6610530614852905)]

In [6]:
glove_wiki_vectors.most_similar("sports")

[('sport', 0.8215380311012268),
 ('soccer', 0.730941891670227),
 ('basketball', 0.7128218412399292),
 ('baseball', 0.7066799402236938),
 ('football', 0.7061923742294312),
 ('sporting', 0.6651638746261597),
 ('entertainment', 0.6645207405090332),
 ('professional', 0.655982494354248),
 ('tv', 0.6407918334007263),
 ('media', 0.6349372863769531)]

In [7]:
glove_wiki_vectors.most_similar("health")

[('care', 0.8693466186523438),
 ('medical', 0.8122366070747375),
 ('healthcare', 0.7658776044845581),
 ('education', 0.7440305948257446),
 ('welfare', 0.7117973566055298),
 ('environmental', 0.7038533687591553),
 ('nutrition', 0.700871467590332),
 ('aids', 0.6989170908927917),
 ('prevention', 0.6923501491546631),
 ('hospitals', 0.6916516423225403)]

In [8]:
glove_wiki_vectors.most_similar("wealth")

[('riches', 0.7063062787055969),
 ('fortune', 0.6760547161102295),
 ('vast', 0.6646038889884949),
 ('enormous', 0.6357147097587585),
 ('income', 0.6310328245162964),
 ('assets', 0.6300343871116638),
 ('immense', 0.6280937790870667),
 ('resources', 0.6276814937591553),
 ('money', 0.6151006817817688),
 ('accumulated', 0.615013837814331)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [9]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [10]:
coast_shore_similarity = glove_wiki_vectors.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = glove_wiki_vectors.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = glove_wiki_vectors.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = glove_wiki_vectors.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = glove_wiki_vectors.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = glove_wiki_vectors.similarity(word_pairs[5][0], word_pairs[5][1])

for i in word_pairs:
    print(i, glove_wiki_vectors.similarity(i[0], i[1]))

('coast', 'shore') 0.7000271
('clothes', 'closet') 0.54627603
('old', 'new') 0.6432488
('smart', 'intelligent') 0.7552733
('dog', 'cat') 0.8798075
('tree', 'lawyer') 0.07671945


<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings

_Points:_ 6

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [13]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [14]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

Yes, there still exist some stereotypes. For instance, white is related to peace while black is associated with conflict; men are associated with leadership while women are associated with party. However, there is not a high similarity between gay and being effeminate or lesbian and being masculine.

In [15]:
analogy("white", "peace", "black")

white : peace :: black : ?


,Analogy word,Score
0,conflict,0.689864
1,unity,0.655489
2,struggle,0.650533
3,reconciliation,0.643959
4,independence,0.641784
5,democracy,0.635591
6,hope,0.626218
7,movement,0.619569
8,accord,0.618663
9,fighting,0.609698


In [16]:
analogy("men", "leader", "women")

men : leader :: women : ?


,Analogy word,Score
0,party,0.735020
1,opposition,0.724580
2,leadership,0.668671
3,president,0.657761
4,democratic,0.650513
5,leaders,0.650468
6,movement,0.644120
7,prime,0.631186
8,candidate,0.628848
9,activist,0.627011


In [17]:
glove_wiki_vectors.similarity("gay", "effeminate")

0.31455043

In [18]:
glove_wiki_vectors.similarity("lesbian", "masculine")

0.34761006

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

_Points:_ 4

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

While the observations I found were quite extreme, the model was able to uncover worrisome biases, especially where the model selected "white : peace :: black : conflict," which represents a significant bias. However, it's worth noting that the model might not be interpreting "white" and "black" as racial categories, but it's still very easy to come to a wrong conclusion. The pre-trained embeddings still exhibit some issues regarding race and gender stereotypes.

A classic example that has been an issue for a long time is AI in predictive policing. The system would preferably hire men as leaders and leave women out. This would cause gender imbalance in the workforce and undermine diversity. It perpetuates discrimination in real world.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 

_Points:_ 8

In lecture 17, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.

In [20]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [21]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [22]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [23]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

In [24]:
from sklearn.metrics import accuracy_score, classification_report

count_vectorizer = CountVectorizer(stop_words='english')
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

lr_bow = LogisticRegression(max_iter=1000)
lr_bow.fit(X_train_bow, y_train)

train_preds_bow = lr_bow.predict(X_train_bow)
test_preds_bow = lr_bow.predict(X_test_bow)

bow_train_accuracy = accuracy_score(y_train, train_preds_bow)
bow_test_accuracy = accuracy_score(y_test, test_preds_bow)
bow_report = classification_report(y_test, test_preds_bow, digits=2) # use digits=2
print(bow_report)

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [26]:
def spacy_avg_vectorizer(docs):
    vectors = []
    for doc in docs:
        tokens = nlp(doc)
        vectors.append(tokens.vector)
    return vectors

X_train_embeddings = spacy_avg_vectorizer(X_train)
X_test_embeddings = spacy_avg_vectorizer(X_test)

lr_emb = LogisticRegression(max_iter=1000)
lr_emb.fit(X_train_embeddings, y_train)

train_preds_emb = lr_emb.predict(X_train_embeddings)
test_preds_emb = lr_emb.predict(X_test_embeddings)

emb_train_accuracy = accuracy_score(y_train, train_preds_emb)
emb_test_accuracy = accuracy_score(y_test, test_preds_emb)
emb_report = classification_report(y_test, test_preds_emb, digits=2) # use digits=2
print(emb_report)

                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.78      0.80       492
enjoy_the_moment       0.57      0.51      0.54       469
        exercise       0.66      0.70      0.68        74
         leisure       0.71      0.65      0.68       407
          nature       0.66      0.72      0.69        71

        accuracy                           0.79      4238
       macro avg       0.73      0.73      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



c:\Users\wsr20\anaconda3\envs\cpsc330\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

_Points:_ 6

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer seperate text by word, so each cell is the frequency of the word appears in text, whereas `avg-emb` is looking for a vector representation in its vector space.
2. CountVectorizer is more interpretable because it returns number of occurrences of a word in the text, whereas `avg-emb` returns a vector in its vector space.
3. Yes. The use of `avg-emb` is transfer learning. I do not observe the benefits of transfer learning since the model trained with `avg-emb` has lower testing scores than `CountVectorizer` one.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [6]:
from sklearn.datasets import fetch_20newsgroups

In [7]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [8]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)

_Points:_ 8

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [2]:
import re
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [3]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """
 
    clean_text = []

    for token in doc:
        if token.like_url or token.like_email:
            continue
        
        elif (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [9]:
def preprocess_text(text):
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"""[\*\~]+""", "", text)
    text = re.sub(r"""[\/]+""", " ", text)

    return text

df["text"] = df["text"].apply(preprocess_text)
df.dropna()

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,It's not a bad question: I don't have any ref...,1,comp.graphics
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics
3,"The following problem is really bugging me, an...",2,comp.windows.x
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone :: I am looking for some software ...,1,comp.graphics
4559,Archive-name: x-faq part3 Last-modified: 1993 ...,2,comp.windows.x
4560,"That's nice, but it doesn't answer the questi...",6,talk.politics.guns
4561,"Hi, I just got myself a Gateway 4DX-33V and tr...",2,comp.windows.x


In [10]:
df["text_pp"] = [
    preprocess(text) for text in nlp.pipe(df["text"])
]

In [11]:
df.head()

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...
1,It's not a bad question: I don't have any ref...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me, an...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification

_Points:_ 2

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

- getting rid of email and urls because they are very unique in their form
- getting rid of slashes and other wired characters
- sentence segmentation and tokenization
- excluding stopwords and punctuation because they are irrelevant when working with data
- lemmatization so that all variants of a word are processed into its root form for better classification and understanding

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

_Points:_ 4

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

Our dataset has 4 general categories of topics, religion, sports, computers, politics.

In [17]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)

from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

lda.fit(X) 
document_topics = lda.transform(X)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

_Points:_ 5

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [18]:
np.argsort(lda.components_, axis=1)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [20]:
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
gun           game          god           image         
people        team          people        file          
file          year          know          55            
state         good          say           format        
right         window        think         jpeg          
armenian      run           come          period        
israel        use           believe       program       
turkish       know          time          graphic       
government    player        thing         color         
entry         play          jesus         play          




- topic 0: politics
- topic 1: sports
- topic 2: religion
- topic 3: computers

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

_Points:_ 5

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [21]:
print(document_topics[0], document_topics[1], document_topics[2], 
      document_topics[3], document_topics[4])

[0.91792255 0.0048727  0.07251588 0.00468886] [0.00368813 0.98887899 0.00376892 0.00366397] [0.00502992 0.00530702 0.83905815 0.15060491] [0.00928782 0.9717597  0.00996268 0.0089898 ] [0.98160245 0.00609432 0.00623438 0.00606885]


The values indicate the probability of the document belonging to each topic. For example, the first  document has a very high probability of belonging to the first topic, and very low probability for the other topics. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

_Points:_ 6

1. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 18 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1. 1. Decision Tree - The process recursively splits the data based on features to create boundaries along the different classes.
   2. KNN - Creates boundaries by looking at the closest classes
   3. Random Forest - Likewise with the decision tree, albeit an ensemble method, a single tree recursively splits the data based on randomly selected features + bootstrapped data to create boundaries along different classes.
   4. Logistic Regression - While binary in nature, Logistic Regression can be extended by using One-vs-Rest or One-vs-One situations. 
   5. SVM RBF - Similar to KNN, it creates boundaries by grouping classes together by using an RBF kernel, which may be using an extension of OvR or OvO.

2. Transfer learning is using parts of a pretrained model in our model to achieve better result.

3. This property is useful for transfer learning because the data is transformed sequentially, which makes it easier to generalize, as well as tailor to different but similar tasks. You could step in and modify one step in the sequence to make the model work for a different task.







<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using [PrairieLearn](https://ca.prairielearn.com/pl/course_instance/6697). Don't forget to rename your file `hw6_sol.ipynb`.

![](img/eva-well-done.png)